Pipeline process for initial CCD processing of images. Uses the *AstroPy* coordinated package `ccdproc`.

In [1]:
## Imports copied from original pipeline, edit as seems fit.
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use("guide.mplstyle")

import numpy as np
from pathlib import Path
import ccdproc as ccdp
from ccdproc import ImageFileCollection
from astropy.stats import mad_std
from astropy import units as u

Specify the working directories. Create target directory if not already present.

In [2]:
# Load cleaned data directory. If no separate directory exists, assume the raw
# data already satisfies the criteria and load from there.
if Path('data-cleaned').exists():
    data_directory = Path('data-cleaned')
else:
    data_directory = Path('data')

# Calibrated data target directory
data_calibrated = Path('data-reduced')
# Make sure this directory exists; exist_ok means this line does nothing if
# already created this folder.
data_calibrated.mkdir(exist_ok=True)

Load images directory

In [3]:
# Load images in ImageFileCollection - convenient way to manage an inventory
# provided by *ccdproc*.
data_collection = ImageFileCollection(data_directory)

## Combine darks for all available exposure times.

In [4]:
# Dark combine - follow the procedure given in the *ccdproc* manual at 
# https://www.astropy.org/ccd-reduction-and-photometry-guide/v/dev/notebooks/03-06-Combine-darks-for-use-in-later-calibration-steps.html


# Find all unique exposure times of dark frames
matches_dark = data_collection.summary['imagetyp'] == 'DARK'
exptime_list = np.unique(data_collection.summary['exptime'][matches_dark])

# Function that produces a master dark given exposure time
def produce_master_dark(exp):
    """Fucntion that produces a master dark given exposure time. Utilises the 
    capabilities of `ccdproc.ImageFileCollection`. The `ccdproc.combine` uses
    parameters recommended in the manual of the package

    Parameters
    ----------
    exp : float or numpy.float64
        Exposure time of darks to be combined in a master dark.
    """
    # Select images: darks with given exptime
    cleaned_darks = data_collection.files_filtered(imagetyp='dark', exptime=exp,
                                                    include_path=True)
    # combine images
    combined_dark = ccdp.combine(cleaned_darks,
                                method='average',
                                sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                mem_limit=350e6
                                )
    # record master dark
    combined_dark.meta['combined'] = True
    dark_file_name = 'combined_dark_{}s.fit'.format(int(exp))
    combined_dark.write(data_calibrated / dark_file_name, overwrite=True)

# Produce a master dark for each unique exposure time
for time in exptime_list:
    produce_master_dark(time)

Load the combined darks to subtract from science (and calibration) images.

In [5]:
# codewords for science images
science_imagetyp = 'light'
calib_imagetyp = 'calib'
exposure = 'exptime'

# Load the collection with master darks and to record to dark subtracted science.
reduced_collection = ccdp.ImageFileCollection(data_calibrated)

Check we loaded images correctly

In [6]:
lights = data_collection.summary[data_collection.summary['imagetyp'] == science_imagetyp.upper()]
lights['date-obs', 'file', 'object', 'filter', exposure]

date-obs,file,object,filter,exptime
str23,str17,str1,str4,float64
2021-08-12T23:05:39.000,vega-180s-001.fit,,None,180.0
2021-08-12T23:08:42.000,vega-180s-002.fit,,None,180.0
2021-08-12T23:11:44.000,vega-180s-003.fit,,None,180.0
2021-08-12T23:14:47.000,vega-180s-004.fit,,None,180.0


In [7]:
combo_calibs = reduced_collection.summary[reduced_collection.summary['combined'].filled(False).astype('bool')]
combo_calibs['date-obs', 'file', 'imagetyp', 'filter', exposure]

date-obs,file,imagetyp,filter,exptime
str23,str22,str5,str4,float64
2022-07-25T22:01:30.000,combined_dark_180s.fit,DARK,None,180.0
2022-07-26T01:21:38.000,combined_dark_5s.fit,DARK,None,5.0


In [8]:
# Dictionary with exposure time -> master dark
combined_darks = {ccd.header[exposure]: ccd for ccd in reduced_collection.ccds(imagetyp='dark', combined=True)}

In [9]:
all_reds_sci = []
light_ccds_sci = []
for light, file_name in data_collection.ccds(imagetyp=science_imagetyp, return_fname=True):
    light_ccds_sci.append(light)
    
    # reduced = ccdp.trim_image(light[:, :4096])
    
    # Note that the first argument in the remainder of the ccdproc calls is
    # the *reduced* image, so that the calibration steps are cumulative.
    # reduced = ccdp.subtract_bias(light, combined_bias)
    
    # closest_dark = find_nearest_dark_exposure(reduced, combined_darks.keys())

    reduced = ccdp.subtract_dark(light, combined_darks[light.header['EXPTIME']], 
                                 exposure_time='exptime', exposure_unit=u.second)
    
    # good_flat = combined_flats[reduced.header['filter']]
    # reduced = ccdp.flat_correct(reduced, good_flat)
    all_reds_sci.append(reduced)
    reduced.write(data_calibrated / file_name, overwrite=True)

In [10]:
calibs = data_collection.summary[data_collection.summary['imagetyp'] == calib_imagetyp.upper()]
calibs['date-obs', 'file', 'object', 'filter', exposure]

date-obs,file,object,filter,exptime
str23,str17,str1,str4,float64
2022-07-25T21:56:36.000,--,,None,5.0
2022-07-25T21:56:44.000,--,,None,5.0
2022-07-25T21:56:52.000,--,,None,5.0


In [11]:
all_reds_cal = []
light_ccds_cal = []
for calib, file_name in data_collection.ccds(imagetyp=calib_imagetyp, return_fname=True):
    light_ccds_cal.append(calib)
    
    # reduced = ccdp.trim_image(light[:, :4096])
    
    # Note that the first argument in the remainder of the ccdproc calls is
    # the *reduced* image, so that the calibration steps are cumulative.
    # reduced = ccdp.subtract_bias(light, combined_bias)
    
    # closest_dark = find_nearest_dark_exposure(reduced, combined_darks.keys())

    reduced = ccdp.subtract_dark(calib, combined_darks[calib.header['EXPTIME']], 
                                 exposure_time='exptime', exposure_unit=u.second)
    
    # good_flat = combined_flats[reduced.header['filter']]
    # reduced = ccdp.flat_correct(reduced, good_flat)
    all_reds_cal.append(reduced)
    reduced.write(data_calibrated / file_name, overwrite=True)